# Mencari kata yang sering muncul menggunakan TF-IDF

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from ast import literal_eval

import pandas as pd
import numpy as np
import json

## Load Data

In [27]:
data = pd.read_csv('../out/01-preprocessing.tsv', sep="\t")
data = data[['raw_content', 'username', 'mention', 'cleaned_content']]
data['mention'] = data['mention'].apply(literal_eval)
data = data.explode('mention', ignore_index=True)

In [23]:
tfidf_vectorizer = TfidfVectorizer()

In [28]:
data_output = []

In [29]:
for username, df in data.groupby(by='username'):
    if len(df) == 1:
        continue
    content = list(df['cleaned_content'].apply(literal_eval))
    mention = list(df['mention'])
    raw_content = list(df['raw_content'])
    tfidf_input = [" ".join(c) for c in content]
    if any(tfidf_input) or all(tfidf_input):
        try:
            tfidf_matrix = tfidf_vectorizer.fit_transform(tfidf_input)
        except:
            print(tfidf_input)
        feature_names = tfidf_vectorizer.get_feature_names_out()
        result = []
        
        for i, doc in enumerate(content):
            feature_index = tfidf_matrix[i, :].nonzero()[1]
            tfidf_scores = zip(feature_index, [tfidf_matrix[i, x] for x in feature_index])
            c_doc = {
                'username': username,
                'mention': mention[i],
                'raw_content': raw_content[i],
                'tf_idf': []
            }
            for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
                c_doc['tf_idf'].append({'token': w, 'score': s})
            result.append(c_doc)
        
        [data_output.append(x) for x in result]

In [30]:
with open('../out/02-tfidf-alt.json', 'w') as f:
    json.dump({'data': data_output}, f, indent=4)